In [2]:
import tensorflow as tf
import os
import re
from collections import Counter

In [6]:
def load_data(file):
    data = []
    with open(file, 'r') as f:
        for line in f:
                line = line.strip()
                label = ' '.join(line[1:line.find("]")].strip().split())
                text = line[line.find("]")+1:].strip()
                data.append([label, text])
    return data
file = 'text.txt'
data = load_data(file)
print("Number of instances: {}".format(len(data)))

Number of instances: 7480


In [9]:
def tokenizer(token, n):
    output = []
    for i in range(n-1, len(token)):
        tokens = ' '.join(token[i-n+1:i+1])
        output.append(tokens)
    return output
def create_feature(text, nrange=(1, 1)):
    text_features = []
    text = text.lower()
    text_alphanum = re.sub('[^a-z)-9#]', ' ', text)
    for n in range(nrange[0], nrange[1]+1):
        text_features += tokenizer(text_alphanum.split(), n)
    text_punc = re.sub('[a-z0-9]', ' ', text)
    text_features += tokenizer(text_punc.split(), 1)
    return Counter(text_features)

In [10]:
def convert_label(item, name): 
    items = list(map(float, item.split()))
    label = ""
    for idx in range(len(items)): 
        if items[idx] == 1: 
            label += name[idx] + " "
    
    return label.strip()
emotions = ["joy", 'fear', "anger", "sadness", "disgust", "shame", "guilt"]
y_all = []
x_all = []
for label, text in data:
    y_all.append(convert_label(label, emotions))
    x_all.append(create_feature(text, nrange=(1, 4)))
count = {}
for i in emotions:
    count[i] = 0
for i in range(len(data)):
    count[y_all[i]] += 1
print(count)

{'joy': 1084, 'fear': 1078, 'anger': 1080, 'sadness': 1079, 'disgust': 1057, 'shame': 1045, 'guilt': 1057}
